# Stock Market Dataset

In [1]:
# Optional if first time use of Kaggle API
# !mkdir -p ~/.kaggle 
# !cp ../kaggle.json ~/.kaggle/
# !chmod 600 ~/.kaggle/kaggle.json

In [2]:
!kaggle datasets download -d  borismarjanovic/price-volume-data-for-all-us-stocks-etfs

price-volume-data-for-all-us-stocks-etfs.zip: Skipping, found more recently modified local copy (use --force to force download)


In [3]:
import os
import random
import zipfile

import pandas as pd
import numpy as np

import datetime
import time
from math import pi

from bokeh.layouts import column
from bokeh.models import ColumnDataSource, RangeTool, HoverTool
from bokeh.plotting import figure, output_notebook, show

In [4]:
if not os.path.isdir('Data'):
    file_name=("price-volume-data-for-all-us-stocks-etfs.zip") 
    with zipfile.ZipFile(file_name,'r') as zip: 
        zip.extractall()
        print("done")

# EDA

Load 10 random company stocks.

In [5]:
os.chdir('Data/Stocks/')

filenames = random.sample([x for x in os.listdir() if x.endswith('.txt') 
                           and os.path.getsize(os.path.join('', x)) > 0], 10)
print(filenames)

['kt.us.txt', 'twmc.us.txt', 'ecol.us.txt', 'rds-a.us.txt', 'arci.us.txt', 'sd.us.txt', 'txn.us.txt', 'ohi.us.txt', 'moh.us.txt', 'dcom.us.txt']


In [6]:
df = []
for i in range(len(filenames)):
    df.append(pd.read_csv(filenames[i], sep=',')[['Date', 'Close']])

In [7]:
df[0].head()

,Date,Close
0,2005-02-25,23.21
1,2005-02-28,23.21
2,2005-03-01,23.20
3,2005-03-02,23.10
4,2005-03-03,22.90


We can see some dates are missing.

In [8]:
df[0]['Date'] = pd.DatetimeIndex(data=df[0]['Date'])
dates = df[0]['Date']

DAY = datetime.timedelta(days=1)
missing = [(d1+DAY, d2) for d1, d2 in zip(dates, dates[1:]) if (d2 - d1) > DAY]

missing[-10:]

[(Timestamp('2017-09-02 00:00:00'), Timestamp('2017-09-05 00:00:00')),
 (Timestamp('2017-09-09 00:00:00'), Timestamp('2017-09-11 00:00:00')),
 (Timestamp('2017-09-16 00:00:00'), Timestamp('2017-09-18 00:00:00')),
 (Timestamp('2017-09-23 00:00:00'), Timestamp('2017-09-25 00:00:00')),
 (Timestamp('2017-09-30 00:00:00'), Timestamp('2017-10-02 00:00:00')),
 (Timestamp('2017-10-07 00:00:00'), Timestamp('2017-10-09 00:00:00')),
 (Timestamp('2017-10-14 00:00:00'), Timestamp('2017-10-16 00:00:00')),
 (Timestamp('2017-10-21 00:00:00'), Timestamp('2017-10-23 00:00:00')),
 (Timestamp('2017-10-28 00:00:00'), Timestamp('2017-10-30 00:00:00')),
 (Timestamp('2017-11-04 00:00:00'), Timestamp('2017-11-06 00:00:00'))]

Most of the missing days are holiday and week-ends: they occur every 5 days and have a span of 2 days. That is when the stock market closes. We should filter the company stock data that we want to use to train our models.

In [9]:
def has_nan(df):
    return df.isnull().values.any()

def has_few_data(df):
    if(df.shape[0] < 2000):
        return True
    
    return False

def use_company_data(df):
    if(has_nan(df)):
        return False
    elif(has_few_data(df)):
        return False
    else:
        return True

Get the list of companies that meet that conditions, which are to have no missing values and to have at least 2000 days of historical data.

In [10]:
files = []
for file in os.listdir():
    try:
        df = pd.read_csv(file, sep=',')
        if(use_company_data( df )):
            files.append(file)
    except:
        continue

len(files)

3429

In [11]:
df = []
for i in range(5):
    df.append(pd.read_csv(files[i], sep=','))

In [12]:
dff = df[0][-50:]
dff["Date"] = pd.to_datetime(dff["Date"])

inc = dff.Close > dff.Open
dec = dff.Open > dff.Close
w = 12*60*60*1000 # half day in ms

TOOLS = "pan,wheel_zoom,box_zoom,reset,save"

p = figure(x_axis_type="datetime", tools=TOOLS, plot_width=1000,
           title = "Candlestick graph for Stock \""+files[0][:-7]+"\"")
p.xaxis.major_label_orientation = pi/4
p.grid.grid_line_alpha=0.3

inc_source = ColumnDataSource(data=dict(
    open1=dff.Open[inc],
    close1=dff.Close[inc],
    Date1=dff.Date[inc]
))

dec_source = ColumnDataSource(data=dict(
    open2=dff.Open[dec],
    close2=dff.Close[dec],
    Date2=dff.Date[dec]
))

p.segment(dff.Date, dff.High, dff.Date, dff.Low, color="black")
r1 = p.vbar(x='Date1', width=w, top='open1', bottom='close1', source=inc_source,
                fill_color="#D5E1DD", line_color="black")
r2 = p.vbar(x='Date2', width=w, top='open2', bottom='close2', source=dec_source,
                fill_color="#F2583E", line_color="black")

# Set up the hover tooltip to display some useful data
p.add_tools(HoverTool(
    renderers=[r1],
    tooltips=[
        ("Open", "$@open1"),
        ("Close", "$@close1"),
        ("Date", "@Date1{%F}"),
    ],
    formatters={
        '@Date1': 'datetime',
    }))

p.add_tools(HoverTool(
    renderers=[r2],
    tooltips=[
        ("Open", "$@open2"),
        ("Close", "$@close2"),
        ("Date", "@Date2{%F}")
    ],
    formatters={
        '@Date2': 'datetime'
    }))

output_notebook()

show(p)

<ipython-input-12-ee19164daded>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dff["Date"] = pd.to_datetime(dff["Date"])


Loading BokehJS ...

In [13]:
dff = df[0]
dff["Date"] = pd.to_datetime(dff["Date"])
dates = dff['Date']
source = ColumnDataSource(data=dict(date=dates, close=dff['Close']))

p = figure(plot_height=300, plot_width=800, tools="xpan", toolbar_location=None,
           x_axis_type="datetime", x_axis_location="above",
           background_fill_color="#efefef", x_range=(dates[dates.index[-100]], dates[dates.index[-1]]))

p.line('date', 'close', source=source)
p.yaxis.axis_label = 'Price'

hover_tool = HoverTool(
    tooltips=[
        ( 'date',   '@date{%F}'            ),
        ( 'close',  '$@{close}{%0.2f}' ), # use @{ } for field names with spaces
    ],

    formatters={
        '@date'        : 'datetime', # use 'datetime' formatter for '@date' field
        '@{close}' : 'printf',   # use 'printf' formatter for '@{adj close}' field
                                     # use default 'numeral' formatter for other fields
    },

    # display a tooltip whenever the cursor is vertically in line with a glyph
    mode='vline'
)
p.add_tools(hover_tool)

TOOLS = "pan,wheel_zoom,box_zoom,reset,save"

select = figure(title="Drag the middle and edges of the selection box to change the range above",
                plot_height=130, plot_width=800, y_range=p.y_range, tools=TOOLS, 
                x_axis_type="datetime", y_axis_type=None
                , toolbar_location=None, background_fill_color="#efefef")

range_tool = RangeTool(x_range=p.x_range)
range_tool.overlay.fill_color = "navy"
range_tool.overlay.fill_alpha = 0.2

select.line('date', 'close', source=source)
select.ygrid.grid_line_color = None
select.add_tools(range_tool, hover_tool)
select.toolbar.active_multi = range_tool

show(column(p, select))

In [14]:
df1 = df[0]
df2 = df[1]
df1["Date"] = pd.to_datetime(df1["Date"])
df2["Date"] = pd.to_datetime(df2["Date"])
dates1 = df1['Date']
dates2 = df2['Date']
source1 = ColumnDataSource(data=dict(date=dates1, close=df1['Close']))
source2 = ColumnDataSource(data=dict(date=dates2, close=df2['Close']))

p = figure(plot_height=300, plot_width=800, tools="xpan", toolbar_location=None,
           x_axis_type="datetime", x_axis_location="above",
           background_fill_color="#efefef", x_range=(dates1[dates1.index[-100]], dates1[dates1.index[-1]]))
tmp = p.x_range # Store it before adding multiple lines
p.line('date', 'close', source=source1, legend_label=files[0][:-7], color='green')
p.line('date', 'close', source=source2, legend_label=files[1][:-7], color='blue')

p.yaxis.axis_label = 'Price'


hover_tool = HoverTool(
    tooltips=[
        ( 'date',   '@date{%F}'            ),
        ( 'close',  '$@{close}{%0.2f}' ), # use @{ } for field names with spaces
    ],

    formatters={
        '@date'        : 'datetime', # use 'datetime' formatter for '@date' field
        '@{close}' : 'printf',   # use 'printf' formatter for '@{adj close}' field
                                     # use default 'numeral' formatter for other fields
    },

    # display a tooltip whenever the cursor is vertically in line with a glyph
    # mode='vline'
)
p.add_tools(hover_tool)

TOOLS = "pan,wheel_zoom,box_zoom,reset,save"

select = figure(title="Drag the middle and edges of the selection box to change the range above",
                plot_height=130, plot_width=800, y_range=p.y_range, tools=TOOLS, 
                x_axis_type="datetime", y_axis_type=None,
                toolbar_location=None, background_fill_color="#efefef")

range_tool = RangeTool(x_range=tmp)
range_tool.overlay.fill_color = "navy"
range_tool.overlay.fill_alpha = 0.2

select.line('date', 'close', source=source)
select.ygrid.grid_line_color = None
select.add_tools(range_tool, hover_tool)
select.toolbar.active_multi = range_tool

show(column(p, select))